## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/TextMining_WordEmbedding

/content/drive/MyDrive/TextMining_WordEmbedding


In [4]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [5]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [6]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [7]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [8]:
corpus[:50]

['12년 전 영국서 찾은 화석 분석… 초식공룡 이구아노돈 뇌조직 판명',
 '공룡의 뇌 화석(사진)이 세계 최초로 확인됐다',
 '데이비드 노먼 영국 케임브리지대 지구과학부 교수팀은 2004년 영국 서식스 주에서 발견된 화석을 분석한 결과 1억3300만 년 전에 살았던 것으로 추정되는 공룡의 뇌 조직인 것으로 밝혀졌다고 27일(현지 시간) 발표했다',
 '\u3000그동안 공룡의 뼈는 화석으로 많이 발견됐지만 뇌 부위는 부드럽고 부패가 빨리 진행돼 화석으로 남아 있는 경우가 없었다',
 '다른 화석으로 추정할 수밖에 없었던 공룡의 뇌 구조를 화석으로 직접 확인했다는 점에서 이번 발견은 의미를 가진다',
 '노먼 교수팀의 공룡 뇌가 화석으로 남아 있을 수 있었던 까닭은 공룡이 사망 직후 산소가 거의 없고 산성도가 높은 물에 머리가 잠겼기 때문이다',
 '일종의 ‘자연적 방부 처리’가 된 셈이다',
 '\u3000노먼 교수팀은 뇌 화석의 주인이 공룡 중에서도 ‘이구아노돈’일 것이라고 추정했다',
 '이구아노돈은 약 1억3300만 년 전, 중생대 백악기에 살았던 육상 공룡으로 몸길이 9m, 몸무게 3t 정도인 초식 공룡이다',
 '카이스트 연구팀 개발, 차세대 저장매체에 응용가능',
 '전기장 인가를 통한 자성 방향의 변화를 나타낸 개념도',
 '자석 아닌 물질에 자성(磁性)을 갖게 하는 기술이 개발됐다',
 '카이스트(KAIST) 물리학과 양찬호 교수 연구팀이 전기장을 통해 자석이 아닌 물질이 자성을 갖게 하거나 그 반대로 자석 내의 자성을 없앨 수 있는 기술을 내놓아 눈길을 끈다',
 '이번 연구를 응용하면 자성 물질 기반의 저장 매체를 개발해 대용량의 정보를 빠른 속도로 이용할 수 있을 것으로 기대된다',
 '물질의 내부에는 아주 작은 자석들이 존재한다',
 '그 작은 자석들이 무질서하게 여러 방향으로 향하고 있으면 비 자성 상태이고 일정한 방향으로 정렬이 이뤄지면 우리가 흔히 볼 수 있는 자석이 된다',
 '정보를 자성 상태로 기록하면 속도가 빠르고 피로 

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]
10개 정도의 문장을 보고 불필요한 단어를 stopwords에 저장한후 tokenizing 했습니다

In [9]:
okt = Okt()

In [86]:
stop_words = '을 이 의 를 에 은 가 는 , 으로 ( 로 수 했다 ) 한 것 하는 에서 과 적 들 할 있다 이다\
              연 등 해 와 기술 인 고 도 된 있는 한다 하고 팀  구 통해 · 다 1 사람 명 말\
              떄 결과 - 이번 그 에는 인간 개 된다 된다 구진 됐다 처럼 에서는 때문 달 성 전 중 보다\
              며 돼 가장 사용 주 3 방식 . 더 되는 다른 일 두 기능 2 이나 체  하면 못 까지 이런 밝혔다\
              했습니다 4 에게 달이 약 대 지 다양한 하 자 제 경우 대한 될 장 방법 간 입니다 이를 같은\
              하지만 지난 단 이라고 뿐 라는 및 했다고 만 하기 위해 아니라 난 에도 고인 속 에서도 작은\
              볼 라고 그러나 데 5 크기 처음 위 있다는 하지 ), 한다는 화 후 공 뒤 조 해야 많은 함께\
              부터 형 게 라 합니다 않고  골 반 있을 양 여러 부사 이후 만들 모두 없는 트 만드는 보인다\
              없이 있어 큰 관련 있습니다 신 상 내 카 음 이는 대해 각 어떤 더욱 가지  달리 안 지금\
              선 하게 다시 리 올해 번 이란 라며 면 따르면 알 에 모습 김 용 부분 시 하면서 하는데 동 따라\
              사이 서로 갖고 Y 삿 잎 서 또 보고 열린 2016 기 = 3일 차 하며 한다고 보즈 면서 결 변 앞\
              했을 14일 있게 또는 초 의해 가능한 D 같다 총 가진 만들어 X 아닌 이고 우리 있기 자기 줄\
              날 엔 하려면 나왔다 필요 일부 도달 나타나는 외 되고 km 사는 생 7 ESA 이라는 않는 곳 출신\
              경 잘 즈 이어 훨씬 타 법 플로 세 되면 9 m 매우 21 세기 있도록 쓸 가능하다 여 건 임\
              있다고 계속 또한 가능할 거 받아 원하는 C 크게 페로 브 보이는 동안 됐습니다 동일한 씩\
              배 값 식 정 뉴호 소 누구 어떻게 갖게 거나 있 힘 뜻 어느 필요하다 돼야 대체 대신\
              력 인해 작 같이 그리고 계 영 제이 다이트 나온 종 않다 없다 낸 대로 그대로 시켜 :\
              < > 나온다 이미 문 우선 부 물론 색 kg 가운데 / 정확하게 구분 층 삼 중이 시켰다\
              찾는 … 빨리 했다는 뿌리는 없고 흔히 최소 셀 하려는 어려웠다 주로 시킬 극 째 혼 A 함 먼은 만을 일어나는 몇 저 어디 스토 나올 로서 않는다 해도 오 호 만들었다'

stop_words = stop_words.split(' ')

stop_words.append(["'", '"', '‘', '“', '”', "’", '`', ")’", ")'"])

In [87]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence):
  tokenized_sentence = []
  sentence = okt.morphs(sentence)
  for word in sentence:
    if not word in stop_words:
      tokenized_sentence.append(word)
  return tokenized_sentence  

In [88]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

In [89]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(min_count=0,min_alpha=0.001)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 4584


In [90]:
print(model_no_pretrained)

Word2Vec(vocab=4584, size=100, alpha=0.025)


In [91]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 5)

(59971, 62555)

In [93]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar('질환')

[('돌연변이', 0.3572320342063904),
 ('1980년', 0.3289148509502411),
 ('컴퓨터', 0.31919461488723755),
 ('아래', 0.31214192509651184),
 ('지나가다가', 0.3031793236732483),
 ('나오고', 0.3005853295326233),
 ('로터스', 0.2952488958835602),
 ('궤도선', 0.2929952144622803),
 ('촬영', 0.2908536195755005),
 ('드라마', 0.28920894861221313)]

In [94]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size = 200, min_count=0)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 4584


In [95]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 32313


In [96]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 5)

(61014, 62555)

In [97]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('폐암')

[('폐렴', 0.8123830556869507),
 ('급성', 0.793777346611023),
 ('고혈압', 0.7819616198539734),
 ('심근', 0.7722629308700562),
 ('심장병', 0.7618685960769653),
 ('간암', 0.7522403001785278),
 ('합병증', 0.7497560977935791),
 ('치매', 0.746001124382019),
 ('백혈병', 0.745927095413208),
 ('결핵', 0.7451514005661011)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]
사전 학습 된 모델이 훨씬 성능이 좋은 것으로 보인다. 파라미터에서 무언가 차이가 나는 것으로 보이는데 사전 학습 없이 학습한 모델에서는 전혀 폐암과 관련된 단어를 찾지 못한다

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

In [98]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

4584

In [99]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_pretrained.wv[i])

In [100]:
pca = PCA(random_state = 0).fit_transform(vector)[:,:3]
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)[:,:2]

In [101]:
print(pca.shape)
print(tsne.shape)

(4584, 3)
(4584, 2)


In [102]:
X, t = make_swiss_roll(n_samples = 1000, noise = 0)
color = (t - t.min()) / (t.max() - t.min())

In [103]:
data = go.Scatter3d(
    x = pca[:, 0],
    y = pca[:, 1],
    z = pca[:, 2],
    text = word,
    marker = dict(
        size = 3,
        colorscale = 'Jet',
        opacity = 0.8
    )
)

layout = go.Layout(
    autosize = False,
    width = 1000,
    height = 1000,
    margin = go.Margin(
        l = 50,
        r = 50,
        b = 100,
        t = 100,
        pad = 4
    )
)

fig = go.Figure(data = [data], layout = layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [119]:
tsne_10000 = TSNE(n_components = 3, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 30000).fit_transform(vector)[:,:3]

In [121]:
data = go.Scatter3d(
    x = tsne_10000[:, 0],
    y = tsne_10000[:, 1],
    z = tsne_10000[:, 2],
    text = word,
    marker = dict(
        size = 3,
        colorscale = 'Jet',
        opacity = 0.8
    )
)

layout = go.Layout(
    autosize = False,
    width = 1000,
    height = 1000,
    margin = go.Margin(
        l = 50,
        r = 50,
        b = 100,
        t = 100,
        pad = 4
    )
)

fig = go.Figure(data = [data], layout = layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [105]:
# t-SNE
tsne_10 = TSNE(n_components = 2, random_state = 0, perplexity = 10, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)[:,:2]

# 시각화
# 시각화해보세요!
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(X.shape[0])],
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [106]:
tsne_5_2000 = TSNE(n_components = 2, random_state = 0, perplexity = 5, learning_rate = 0.05, init = 'random', n_iter = 2000).fit_transform(vector)[:,:2]


In [ ]:

data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(X.shape[0])],
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()


# 수고하셨습니다!